In [1]:
from pathlib import Path
import pandas as pd

In [2]:
tanno_dir = Path('/home/yutanagano/UCLOneDrive/MBPhD/data/tcr_embedder/data_sources/tanno_processed')

In [3]:
example = pd.read_csv(
    tanno_dir/'data'/'A1 memory.txt',
    sep='\t'
)

In [4]:
example.head()

,Clustered,Collapsed,ClusterID,CDRH3_AA,CDRL3_AA,CDRH3_NT,CDRL3_NT,VH,DH,JH,VL,JL
0,8899,5544,1,CASSLQGARETQYF,CALLSTGGGNKLTF,TGTGCCAGCAGCTTACAGGGGGCGCGGGAGACCCAGTACTTC,TGTGCTCTCCTTTCCACGGGAGGAGGAAACAAACTCACCTTT,TRBV11-2,TRBD1,TRBJ2-5,TRAV9-2,TRAJ10
1,1945,1077,2,CSWKREIVEQFF,CAVSDRNTNAGKSTF,TGCAGTTGGAAGCGGGAGATAGTTGAGCAGTTCTTC,TGTGCTGTGAGTGATAGGAACACCAATGCAGGCAAATCAACCTTT,TRBV20-1,TRBD2,TRBJ2-1,TRAV8-4,TRAJ27
2,1215,867,3,CASSSQGANTEAFF,CIVRKRNQFYF,TGTGCCAGCAGCTCCCAAGGAGCGAACACTGAAGCTTTCTTT,TGCATCGTCAGAAAACGTAACCAGTTCTATTTT,TRBV9,NaN,TRBJ1-1,TRAV26-1,TRAJ49
3,1185,866,4,CASSFQGQGGQPQHF,CAVRPRDTGGFKTIF,TGTGCCAGCAGTTTCCAGGGACAGGGCGGTCAGCCCCAGCATTTT,TGTGCTGTGAGGCCTCGGGATACTGGAGGCTTCAAAACTATCTTT,TRBV12-3,TRBD1,TRBJ1-5,TRAV21,TRAJ9
4,1002,785,5,CSAPLAGVSYNEQFF,CIVRSYNYGQNFVF,TGCAGTGCGCCACTAGCGGGCGTCTCCTACAATGAGCAGTTCTTC,TGCATCGTCAGATCTTATAACTATGGTCAGAATTTTGTCTTT,TRBV20-1,TRBD2,TRBJ2-1,TRAV26-1,TRAJ26


In [5]:
dfs = []

for f in (tanno_dir/'data').iterdir():
    df = pd.read_csv(f,sep='\t')
    df = df[
        ['VL', 'CDRL3_AA', 'JL', 'VH', 'CDRH3_AA', 'JH']
    ]
    df = df.drop_duplicates()
    df.columns = ['TRAV', 'CDR3A', 'TRAJ', 'TRBV', 'CDR3B', 'TRBJ']
    df['Epitope'] = pd.NA
    df['MHCA'] = pd.NA
    df['MHCB'] = pd.NA
    df['duplicate_count'] = pd.NA

    dfs.append(df)

combined = pd.concat(dfs, axis='index')
combined = combined.drop_duplicates()
combined = combined.sort_values(
    by=combined.columns.tolist(),
    ignore_index=True
)

In [6]:
combined.head()

,TRAV,CDR3A,TRAJ,TRBV,CDR3B,TRBJ,Epitope,MHCA,MHCB,duplicate_count
0,TRAV1-1,CAAANQAGTALIF,TRAJ15,TRBV6-1,CASSDGVAETQYF,TRBJ2-5,<NA>,<NA>,<NA>,<NA>
1,TRAV1-1,CAAANTGFQKLVF,TRAJ8,TRBV27,CASSPLQGYEQYF,TRBJ2-7,<NA>,<NA>,<NA>,<NA>
2,TRAV1-1,CAAAPGFQKLVF,TRAJ8,TRBV7-2,CASSPTVSQETQYF,TRBJ2-5,<NA>,<NA>,<NA>,<NA>
3,TRAV1-1,CAAAQAGTALIF,TRAJ15,TRBV30,CAWRDVAGAEKLFF,TRBJ1-4,<NA>,<NA>,<NA>,<NA>
4,TRAV1-1,CAAAQGGKLIF,TRAJ23,TRBV7-9,CASTGSNYGYTF,TRBJ1-2,<NA>,<NA>,<NA>,<NA>


In [7]:
combined.to_csv(tanno_dir/'combined.csv', index=False)

In [8]:
test_frac = 0.1
test_rows = int(test_frac * len(combined))

shuffled = combined.sample(frac=1)

test = shuffled.iloc[:test_rows]
train = shuffled.iloc[test_rows:]

In [9]:
train.to_csv(tanno_dir/'train.csv', index=False)
test.to_csv(tanno_dir/'test.csv', index=False)